In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.14.0


In [0]:
import pandas as pd
dataset=pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3)

In [0]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [0]:
vocab_size = 1000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

W0709 03:57:39.485189 140540506912640 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0709 03:57:39.531183 140540506912640 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 16,289
Trainable params: 16,289
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

W0709 03:58:04.887394 140540506912640 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
history = model.fit(X_train,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(X_test, y_test),
                    verbose=1)

Train on 800 samples, validate on 200 samples
Epoch 1/40
800/800 [==============================] - 1s 819us/sample - loss: 0.6932 - acc: 0.4963 - val_loss: 0.6931 - val_acc: 0.5150
Epoch 2/40
800/800 [==============================] - 0s 237us/sample - loss: 0.6932 - acc: 0.4963 - val_loss: 0.6931 - val_acc: 0.5150
Epoch 3/40
800/800 [==============================] - 0s 236us/sample - loss: 0.6932 - acc: 0.4963 - val_loss: 0.6932 - val_acc: 0.4850
Epoch 4/40
800/800 [==============================] - 0s 241us/sample - loss: 0.6931 - acc: 0.5038 - val_loss: 0.6932 - val_acc: 0.4850
Epoch 5/40
800/800 [==============================] - 0s 233us/sample - loss: 0.6931 - acc: 0.5038 - val_loss: 0.6933 - val_acc: 0.4850
Epoch 6/40
800/800 [==============================] - 0s 244us/sample - loss: 0.6931 - acc: 0.5038 - val_loss: 0.6933 - val_acc: 0.4850
Epoch 7/40
800/800 [==============================] - 0s 241us/sample - loss: 0.6931 - acc: 0.5038 - val_loss: 0.6933 - val_acc: 0.4850
Ep

In [0]:
results = model.evaluate(X_test,y_test)

print(results)

200/200 [==============================] - 0s 102us/sample - loss: 0.6934 - acc: 0.4850
[0.6933714747428894, 0.485]
